[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rlwPIErsvGcYYgM6s_FT9FDn9lcHzvVU?usp=sharing)

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/git/fer-tar

/content/gdrive/.shortcut-targets-by-id/1yfuNPQUT_G0CfNtdGxHhLi96giKyswbZ/git/fer-tar


In [ ]:
from importlib import reload

# 1. Train GPT augmenter

In [ ]:
from my_scripts.GPTAug.data.train2 import *
import my_scripts.GPTAug.data.train2
reload(my_scripts.GPTAug.data.train2)

ModuleNotFoundError: ignored

In [ ]:
BATCH_SIZE = 32
EPOCHS = 4
LEARNING_RATE = 5e-5
WARMUP_STEPS = 300
MAX_SEQ_LEN = 32
MODEL_NAME = 'gpt_medium_double_aug'

DATA_FILE = '/content/gdrive/MyDrive/git/fer-tar/my_scripts/GPTAug/stvarno_ne_znam.csv'

TOKENIZER, MODEL = load_models()


LOADER = get_data_loader(DATA_FILE)
DEVICE = 'cpu'
if torch.cuda.is_available():
    DEVICE = 'cuda'

model = MODEL.to(DEVICE)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)

model = train(EPOCHS, LOADER, BATCH_SIZE, TOKENIZER, MODEL, DEVICE, optimizer, scheduler)
save_model(model, MODEL_NAME)

# 2. Generate data

In [ ]:
import my_scripts.GPTAug.data.generate
reload(my_scripts.GPTAug.data.generate)
from my_scripts.GPTAug.data.generate import *

In [ ]:
SENTENCES = 30000
MODEL_NAME = 'gpt_medium_double_aug.pt'
LABEL = '0'

# TOKENIZER, MODEL = load_models(MODEL_NAME)
generate(TOKENIZER, MODEL, SENTENCES)

100%|██████████| 30000/30000 [2:57:18<00:00,  2.82it/s]


In [ ]:
M = MODEL

# 3. Cleanup

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('GPT2_medium_aug_sense4.csv')
data.head()

,id,sentence
0,0,I put the bread in the oven/I put the fire in ...
1,1,the doctor has been sent home for the treatmen...
2,2,He ate some tuna./He ate lots of steel.
3,3,She uses her hands to eat bananas/She uses her...
4,4,I want to get married/I want to get eaten


In [ ]:
!pip install lm-scorer
from lm_scorer.models.auto import AutoLMScorer
import torch

     |████████████████████████████████| 1.6MB 13.6MB/s 
     |████████████████████████████████| 675kB 55.3MB/s 
     |████████████████████████████████| 901kB 52.8MB/s 
     |████████████████████████████████| 5.6MB 27.4MB/s 
     |████████████████████████████████| 1.2MB 58.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
data['sentence'].replace('\n.*', '', regex=True, inplace=True)

In [ ]:
data = data[~data['sentence'].str.contains('^.*/.*/.*$')]

In [ ]:
data = data[data['sentence'].str.contains('/')]

In [ ]:
data[['sent0', 'sent1']] = data['sentence'].str.split('/', 1, expand=True)

In [ ]:
data = data[data['sent0'] != data['sent1']]

In [ ]:
data['label'] = np.random.randint(0, 2, size=(len(data)))

In [ ]:
data['sent0'], data['sent1'] = np.where(data['label'] == 0, (data['sent1'], data['sent0']), (data['sent0'], data['sent1']))

In [ ]:
labels = pd.DataFrame(data['label'])
sentences = pd.DataFrame(data[['sent0', 'sent1']])

In [ ]:
labels.to_csv('GPT2_answers.csv', header=None)
sentences.to_csv('GPT2_subtaskA_data.csv', index_label='id')